In [48]:
import numpy as np
import re
import pandas as pd
import os
import codecs
os.environ["PATH"] += os.pathsep + 'D:/Program Files (x86)/Graphviz2.38/bin/' 

In [49]:
from gensim.models.word2vec import Word2Vec
from gensim.models import KeyedVectors
from gensim import utils
from stop_words import get_stop_words
import nltk
import string
from nltk.corpus import stopwords
from tqdm import tqdm
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import normalize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet as wn
import pymorphy2
import pymystem3

In [50]:
df_neg = pd.read_csv('negative.csv', delimiter=';',header=None )
df_pos = pd.read_csv('positive.csv', delimiter=';',header=None )

In [51]:
df_neg.columns = ['id','tdate','tmane','ttext','ttype','trep','trtw','tfav','tstcount','tfol','tfrien','listcount']
df_pos.columns = ['id','tdate','tmane','ttext','ttype','trep','trtw','tfav','tstcount','tfol','tfrien','listcount']

In [52]:
df_neg.head()

,id,tdate,tmane,ttext,ttype,trep,trtw,tfav,tstcount,tfol,tfrien,listcount
0,408906762813579264,1386325944,dugarchikbellko,на работе был полный пиддес :| и так каждое за...,-1,0,0,0,8064,111,94,2
1,408906818262687744,1386325957,nugemycejela,"Коллеги сидят рубятся в Urban terror, а я из-з...",-1,0,0,0,26,42,39,0
2,408906858515398656,1386325966,4post21,@elina_4post как говорят обещаного три года жд...,-1,0,0,0,718,49,249,0
3,408906914437685248,1386325980,Poliwake,"Желаю хорошего полёта и удачной посадки,я буду...",-1,0,0,0,10628,207,200,0
4,408906914723295232,1386325980,capyvixowe,"Обновил за каким-то лешим surf, теперь не рабо...",-1,0,0,0,35,17,34,0


In [53]:
df_pos.head()

,id,tdate,tmane,ttext,ttype,trep,trtw,tfav,tstcount,tfol,tfrien,listcount
0,408906692374446080,1386325927,pleease_shut_up,"@first_timee хоть я и школота, но поверь, у на...",1,0,0,0,7569,62,61,0
1,408906692693221377,1386325927,alinakirpicheva,"Да, все-таки он немного похож на него. Но мой ...",1,0,0,0,11825,59,31,2
2,408906695083954177,1386325927,EvgeshaRe,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,0,1,0,1273,26,27,0
3,408906695356973056,1386325927,ikonnikova_21,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,0,1,0,1549,19,17,0
4,408906761416867842,1386325943,JumpyAlex,@irina_dyshkant Вот что значит страшилка :D\nН...,1,0,0,0,597,16,23,1


In [67]:
df =pd.concat((df_pos, df_neg), axis=0)

In [68]:
y =df.ttype

In [69]:
del df['ttype']

In [70]:
df.drop(['id', 'tdate'], inplace=True)

In [71]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 226834 entries, 0 to 111922
Data columns (total 11 columns):
id           226834 non-null int64
tdate        226834 non-null int64
tmane        226834 non-null object
ttext        226834 non-null object
trep         226834 non-null int64
trtw         226834 non-null int64
tfav         226834 non-null int64
tstcount     226834 non-null int64
tfol         226834 non-null int64
tfrien       226834 non-null int64
listcount    226834 non-null int64
dtypes: int64(9), object(2)
memory usage: 20.8+ MB


In [72]:
df.tmane.nunique()

139658

In [73]:
df.drop(['tmane'], inplace=True)

In [61]:
import matplotlib
import seaborn as sns

In [66]:
%config InlineBackend.figure_format = 'png' 
sns_plot = sns.pairplot(
    df);
sns_plot.savefig('pairplot.png')

In [ ]:
class Morpher():
    '''
    activation - distance function betven labeldata and answer
    '''

    def __init__(self, model_rucopr,mystem, pymorphy ):
        self.model_rucopr=model_rucopr
        self.mystem = mystem
        self.pymorph = pymorphy
        self.count=0
    def normality_me(self,query):
        if (self.count%50000) ==0:
            print(self.count)
        self.count+=1
        penn_to_wn = { 
            'S': wn.NOUN, 
            'V': wn.VERB, 
            'ADV': wn.ADV, 
            'A': wn.ADJ}
        pym_to_rutag ={
            'S': 'NOUN',  
            'V': 'VERB',
            'ADV': 'ADV', 
            'A': 'ADJ'  }          
    
#        tags = [(morph.parse(word)[0].word, morph.parse(word)[0].tag.POS) for word in  word_tokenize(query)]
        
#         tmpinput = codecs.open('tmpinput', 'w', 'utf-8') 
#         tmpinput.write(query)
#         tmpinput.close()
#         tmpoutput = codecs.open('tmpoutput', 'w', 'utf-8')
#         tmpoutput.close()
#         !mystem -dnil -e utf-8 tmpinput tmpoutput
#         tmpoutput = codecs.open('tmpoutput', 'r', 'utf-8')
#         lines = tmpoutput.readlines()
#         tmpoutput.close()
#         words = self.mystem.lemmatize(query)


        words = [self.pymorph.parse(s)[0][2] for s in  word_tokenize(query)]
        return ' '.join(words)

        patternnotvalid = re.compile('.*\?.*')
        tags = [(tstres[11]['analysis'][0]['gr'].split(',')[0],tstres[11]['analysis'][0]['lex']) for line in lines if not bool(patternnotvalid.match(tstres[11]['analysis'][0]['lex'])) ]

                                            
        words=[]
        for tag in tags:
            wn_tag = penn_to_wn.get(tag[1], wn.NOUN)
            word = morph.normal_forms(tag[0])[0]
            words.append((WordNetLemmatizer().lemmatize(word,wn_tag).lower() + "_" + pym_to_rutag.get(tag[1], 'NOUN')))
        return " ".join(words)
    def text_to_vec(self, text):
        if (len(text)==0) | (text is None):
            return (np.zeros((1,300)), '', '' , '')
        if 1==1:
            pattern= re.compile('[^А-Яа-я\s]+')
            pattern_spaces = re.compile('\s{1,}')
            text = pattern_spaces.sub(' ', pattern.sub(' ', text))
            text = self.normality_me(text)
            
            words = text.split(' ')
            words_err = []
            words_true = []
            sumvec = np.zeros((300,))
            
            for word in words:
                try:
                    vec = self.model_rucopr.word_vec(word)
                    sumvec = sumvec + vec
                    words_true.append(word)
                except:
                    words_err.append(word)
        return(normalize(sumvec.reshape(1, -1)), words, words_true, words_err)

In [17]:
model_rucopr = KeyedVectors.load_word2vec_format('D:\home\datasets\\ruwikiruscorpora_0_300_20.bin', binary=True)

In [18]:
pymorph = pymorphy2.MorphAnalyzer()
mystem = pymystem3.Mystem()

In [19]:
pymorph.parse('мыла')[0][2]

'мыло'

In [20]:
mystem.lemmatize('Мама мыла раму') ## намного лучше но оооочень долго, по прогнозам больше 20 часов

['мама', ' ', 'мыть', ' ', 'рама', '\n']

In [21]:
mymorph = Morpher(model_rucopr,mystem,pymorph )

In [22]:
%%time
testlemotise=[mymorph.normality_me(s) for s in df.ttext.head(10)]

0
Wall time: 3.85 s


In [23]:
testlemotise

['@ first_timee хоть я и школотый , но поверь , у мы то же самый : d общество профилировать предмет тип )',
 'да , всё-таки он немного похожий на он . но мой мальчик весь равно хороший : d',
 'rt @ katiacheh : ну ты идиотка ) я испугаться за ты ! ! !',
 "rt @ digger2912 : `` кто то в угол сидеть и погибать от голод , а мы ещё 2 порция взять , хотя уже и так жрать не хотеть '' : dd http : //t.co/gqg6iue2…",
 '@ irina_dyshkant вот что значит страшилка : d но блин , посмотреть весь часть , у ты создаться ощущение , что автор курить что-то : d',
 'ну любить или нет ? — я не знать кто ты бля : d http : //t.co/brf9eng1u6',
 'rt @ spoonlamer : ох,900 : d ну это конечно же @ twinkleayo . чтобы у нея быть много друг , ведь она такой мимими & lt ; 3',
 'rt @ veregijytaqo : у ты есть ухажёр ? нет - мой ухо не кто не жрать : d',
 'поприветствовать мой новое читатель @ alexey1789 ; )',
 'теперь у я есть частичка сидней : ) # sydney # sydneyoperahouse http : //t.co/agnto3czei']

In [42]:
re.sub('\s\d+\s', '$number$', re.sub('[\(\)\s09(?:\:d+)]+', ' ', re.sub(  '@[A-Za-z\_\d]+', '$username$', 
                                                                                '$username$ Вот что значит страшилка :D\n yj'), re.IGNORECASE))

'$username$ Вот что значит страшилка :D\n yj'

In [75]:
pattern_user=re.compile('@[A-Za-z\_\d]+')
pattern_num = re.compile('\s\d+\s')
pattern_smile = re.compile('[\(\)\s09(?:\:D+)(?:\:d+)]+')

In [80]:
def preproc(s):
    s=pattern_user.sub(' $username$ ', s)
    s=pattern_smile.sub(' ', s)
    s=pattern_num.sub(' $number$ ', s)
    return s

In [81]:
df.ttext = [preproc(s) for s in df.ttext]

In [82]:
df.ttext.head()

0     $username$ хоть я и школота, но поверь, у нас...
1    Да, все-таки он немного похож на него. Но мой ...
2    RT $username$ Ну ты идиотка я испугалась за те...
3    RT $username$ "Кто то в углу сидит и погибает ...
4     $username$ Вот что значит страшилка Но блин,п...
Name: ttext, dtype: object

In [84]:
%%time
df['ttext_norm'] = [mymorph.normality_me(s) for s in df.ttext ]

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
Wall time: 3h 25min 20s


In [85]:
import pickle

In [86]:
file = open('df', 'wb')
pickle.dump(df,file)
file.close()

In [87]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler

In [88]:
y_tr, y_val, x_tr,  x_val = train_test_split(y, df, test_size=0.2, random_state=42 )

In [89]:
text = x_tr.ttext_norm

In [90]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [91]:
from nltk.corpus import stopwords

In [92]:
stopwords.words('russian')[:10]

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со']

In [93]:
countvec = CountVectorizer(min_df= 5, stop_words= stopwords.words('russian'),
                            preprocessor= lambda s: re.sub('http',' ', s) 
                          )

In [94]:
countvec2 = CountVectorizer(min_df= 5, stop_words= stopwords.words('russian'),
                            preprocessor= lambda s: re.sub('http',' ', s) 
                          )

In [95]:
tfidf = TfidfVectorizer(min_df= 5, stop_words= stopwords.words('russian'))

In [96]:
tfidf2 = TfidfVectorizer(min_df= 5, stop_words= stopwords.words('russian'))

In [97]:
%%time
data_cv      = countvec.fit_transform(x_tr.ttext)
data_cv_norm = countvec2.fit_transform(x_tr.ttext_norm)

Wall time: 11.4 s


In [98]:
%%time
data_tf      = tfidf.fit_transform(x_tr.ttext)
data_tf_norm = tfidf2.fit_transform(x_tr.ttext_norm)

Wall time: 10.8 s


In [99]:
ss_cv = StandardScaler(with_mean=False)
ss_cv_norm = StandardScaler(with_mean=False)
ss_tf = StandardScaler(with_mean=False)
ss_tf_norm = StandardScaler(with_mean=False)

In [100]:
data_cv_ss=ss_cv.fit_transform(data_cv)
data_cv_norm_ss=ss_cv_norm.fit_transform(data_cv_norm)
data_tf_ss = ss_tf.fit_transform(data_tf)
data_tf_norm_ss = ss_tf_norm.fit_transform(data_tf_norm)

D:\distr\anaconda\lib\site-packages\sklearn\utils\validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)


In [101]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

In [102]:
logreg = LogisticRegression(C=0.0001)

In [103]:
from sklearn.model_selection import cross_val_score

In [104]:
res={}

In [105]:
res['cv'] = np.mean(cross_val_score(logreg, data_cv, y_tr, cv = 5, scoring='accuracy'))

In [106]:
res['cv_ss'] = np.mean(cross_val_score(logreg, data_cv_ss, y_tr, cv = 5, scoring='accuracy'))

In [107]:
res['cv_norm_ss'] = np.mean(cross_val_score(logreg, data_cv_norm_ss, y_tr, cv = 5, scoring='accuracy'))

In [108]:
res['tf'] = np.mean(cross_val_score(logreg, data_tf, y_tr, cv = 5, scoring='accuracy'))

In [109]:
res['tf_ss'] = np.mean(cross_val_score(logreg, data_tf_ss, y_tr, cv = 5, scoring='accuracy'))

In [110]:
res['tf_norm_ss'] = np.mean(cross_val_score(logreg, data_tf_norm_ss, y_tr, cv = 5, scoring='accuracy'))

In [111]:
res

{'cv': 0.5974143821334017,
 'cv_norm_ss': 0.72164635558762802,
 'cv_ss': 0.72037891024419465,
 'tf': 0.50950861440820439,
 'tf_norm_ss': 0.71600896223148369,
 'tf_ss': 0.71595382117394768}

### как мы видим нормализация помогла совсем чуть-чуть, больше буста дал - StandartScaler 
### tfidf чуть хуже CountVec в данном случае, думаю потому, что много опечаток, и тексты короткие

In [112]:
logreg.fit(data_cv_norm_ss, y_tr)

LogisticRegression(C=0.0001, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [115]:
logreg.coef_[0,np.argsort(logreg.coef_)]

array([[-0.14785172, -0.12358586, -0.11852034, ...,  0.11628097,
         0.15191541,  0.18309084]])

In [116]:
features = np.array(countvec2.get_feature_names())

In [117]:
arr= np.abs(logreg.coef_)

NameError: name 'logreg2' is not defined

In [430]:
indexes = arr.argsort()

In [431]:
indexes[0,-20:][::-1]

array([21639,     0, 21638,     1, 21637,     2,     3,     4,     5,
           6,     7, 21636, 21635,     8, 21634,     9,    10, 21633,
          11,    12], dtype=int64)

In [432]:
logreg2.coef_[0,indexes[0,-20:][::-1]]

array([ 0.18139612, -0.14777354,  0.13342191, -0.12512714,  0.12282642,
       -0.11866454, -0.11834752, -0.11273263, -0.10497601, -0.10066051,
       -0.09639302,  0.09156947,  0.08878459, -0.08820877,  0.08703229,
       -0.08372495, -0.08333463,  0.07566398, -0.07232287, -0.07065004])

In [433]:
features[indexes[0,-20:][::-1]]

array(['өөрөө', '00', 'өөрсдийгөө', '000', 'өөрийгөө', '0000',
       '000nana000', '00darya', '00okaay', '01', '02', 'өөр', 'өө', '03',
       'өнөөдөр', '04', '05', 'өмнө', '06', '07'],
      dtype='<U33')

In [435]:
feature_imp = np.vstack((features[indexes[0,-20:][::-1]],    logreg2.coef_[0,indexes[0,-20:][::-1]]))

In [436]:
feature_imp.T

array([['өөрөө', '0.18139611783258958'],
       ['00', '-0.14777354205522386'],
       ['өөрсдийгөө', '0.13342190647512253'],
       ['000', '-0.12512713594840152'],
       ['өөрийгөө', '0.12282641567570128'],
       ['0000', '-0.11866454324548698'],
       ['000nana000', '-0.11834752316401838'],
       ['00darya', '-0.11273263373450153'],
       ['00okaay', '-0.1049760075425699'],
       ['01', '-0.10066050547392473'],
       ['02', '-0.09639302225164473'],
       ['өөр', '0.09156946634552089'],
       ['өө', '0.08878458992383059'],
       ['03', '-0.08820876647269975'],
       ['өнөөдөр', '0.08703228719052573'],
       ['04', '-0.08372494731861087'],
       ['05', '-0.08333463253976603'],
       ['өмнө', '0.07566397929019616'],
       ['06', '-0.07232286860019824'],
       ['07', '-0.07065003691034456']],
      dtype='<U33')

In [297]:
from  sklearn.metrics import confusion_matrix

In [298]:
data_cv_norm_ss_val = ss_cv_norm.transform(countvec2.transform(x_val.ttext_norm))

D:\distr\anaconda\lib\site-packages\sklearn\utils\validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)


In [306]:
prediction_cv_norm_ss_val= logreg.predict(data_cv_norm_ss_val)

In [307]:
confusion_matrix(prediction_cv_norm_ss_val, y_val)

array([[16181,  5645],
       [ 6299, 17242]])